# Flatmap data

This notebook creates plots of the polar and eccentricity data that was projected to the flatmap by the SUIT toolbox.

In [ ]:
import numpy as np
import pandas as pd
import pycircstat as pc
import copy as copy
import os
import nibabel as nb
import statsmodels.stats.api as sms


import matplotlib.pyplot as pl
import seaborn as sn
sn.set_style('ticks')
import glob

import matplotlib as mpl
%matplotlib inline
mpl.rc_file_defaults()

### define directories

In [ ]:
# define directories
data_dir = '/home/shared/2018/visual/hcp_cerebellum/'
repo_dir  = '/home/vanes/git/hcp_cerebellum/'
suit_home  = '/home/vanes/bin/suit/'

### subdirectories

In [ ]:
resource_dir = os.path.join(repo_dir,'resources')
flat_dir = os.path.join(data_dir,'flat_data')

# setup figure directory
fig_dir = os.path.join(data_dir,'figs')
if not os.path.isdir(fig_dir): os.mkdir(fig_dir)

In [ ]:
# these are the coordinates of the SUIT flat data:
# in the toolbox, these are located in $spmhome/suit/flatmap
coords=nb.load(os.path.join(suit_home,'flatmap','FLAT.coord.gii'))

x = np.ravel(coords.get_arrays_from_intent(1008)[0].data[:,0]) 
y = np.ravel(coords.get_arrays_from_intent(1008)[0].data[:,1]) 

In [ ]:
# get the flatmap representations of the retmaps
masks=np.ravel(pd.read_csv(os.path.join(flat_dir,'retmaps_flat.csv'),header=None))
sjs = ['181','181_1','181_2','182','182_1','182_2','183','183_1','183_2']



In [ ]:
roi_order = ['left_OMV','right_OMV','left_VIIb','right_VIIb','left_VIIIb','right_VIIIb']

these_combs = {
    'left_OMV':['left_mOMV','left_lOMV'],
    'right_OMV':['right_mOMV','right_lOMV'],
    'left_VIIb':['left_VIIb'],
    'right_VIIb':['right_VIIb'],
    'left_VIIIb':['left_mVIIIb','left_lVIIIb'],
    'right_VIIIb':['right_mVIIIb','right_lVIIIb'],    
}

In [ ]:
def rotate_via_numpy(x,y, radians):
    """Use numpy to build a rotation matrix and take the dot product."""
    c, s = np.cos(radians), np.sin(radians)
    j = np.matrix([[c, s], [-s, c]])
    m = np.dot(j, np.matrix([x, y]))

    return m

In [ ]:
def get_roi_color(mask):
    
    # determine base roi colors
    if mask in ['left_OMV','right_OMV','OMV']:
        c = '#6590CB'
    elif mask in ['left_VIIIb','right_VIIIb','VIIIb']:
        c = '#E55D5C'
    elif ('VIIb' in mask) + (mask=='VIIb'):
        c = '#E79F2A'
        
    # roi colors for lateral / medial separation:
    elif mask in ['left_lOMV','right_lOMV','lOMV']:
        c = '#6590CB'
    elif mask in ['left_mOMV','right_mOMV','mOMV']:
        c = '#0D5B99'
    elif mask in ['left_mVIIIb','right_mVIIIb','mVIIIb']:
        c = '#C03142'
    elif mask in ['left_lVIIIb','right_lVIIIb','lVIIIb']:
        c = '#E57F80'
        
    return c

In [ ]:
# these refer to the indices in cerebellum_retmaps.nii
mask_names ={
'left_mOMV':7,
'right_mOMV':9,
'left_lOMV':8,
'right_lOMV':10,
    
'left_VIIb':5,
'right_VIIb':6,
    
'left_mVIIIb':3,
'right_mVIIIb':2,
'left_lVIIIb':4,
'right_lVIIIb':1,
}


# this is how the lateral/medial and left right hemisphere indices combine:
roi_combs = {
    'left_OMV':[7,8],
    'right_OMV':[9,10],
    'left_VIIb':[5],
    'right_VIIb':[6],
    'left_VIIIb':[3,4],
    'right_VIIIb':[1,2],

    'OMV':[7,8,9,10],
    'lOMV':[8,10],
    'mOMV':[7,9],
    'VIIb':[5,6],
    'VIIIb':[1,2,3,4],
    'lVIIIb':[1,4],
    'mVIIIb':[2,3],
}

roi_order = ['left_OMV','right_OMV','left_VIIb','right_VIIb','left_VIIIb','right_VIIIb']
# roi_comb_order = ['mOMV','lOMV','VIIb','mVIIIb','lVIIIb']           

In [ ]:
kernel = 6 # 1/kernel is the proportion of data in single moving average operation

# angles
angle_rot_mask = {
    'left_OMV':0.75,
    'right_OMV':0.25,
    'left_VIIb':1.25,
    'right_VIIb':1.75,
    'left_VIIIb':0.75,
    'right_VIIIb':0.25,

}    

# extents
angle_roi_extents = {
    'left_OMV':[14,32],
    'right_OMV':[14,33],
    'left_VIIb':[5,40],
    'right_VIIb':[3,37],
    'left_VIIIb':[-13,30],
    'right_VIIIb':[-12,18],
}   

# angles
ecc_rot_mask = {
    'left_OMV':0.25,
    'right_OMV':0.75,
    'left_VIIb':0,
    'right_VIIb':1,
    'left_VIIIb':0.25,
    'right_VIIIb':0.75,
}

# extents
ecc_roi_extents = {
    'left_OMV':[6,17],
    'right_OMV':[4,15],
    'left_VIIb':[-28,-2],
    'right_VIIb':[-27,-1],
    'left_VIIIb':[-90,-55],
    'right_VIIIb':[-87,-57],

}   

In [ ]:
def weighted_moving_avg_ang(x_data,y_data,weights,kernel=6):

    """
    this function was only used in initial version of manuscript
    """
    
    # first sort values based on x
    order = np.argsort(x_data)
    xo = x_data[order]
    yo = y_data[order]
    wo = weights[order]
    n = len(weights)
    kw = int(n/kernel)
    
    # initialize variables
    ma_x = []
    ma_y = []
    ma_y_ci = []
    # loop over bins
    for i in range(n-kw):
        valid_vertices = np.arange(len(wo))[i:i+kw]
        valid_vertices = valid_vertices[~np.isnan(wo[valid_vertices])]
        try:
            ma_y_ci.append(pc.mean_ci_limits(yo[valid_vertices],w=wo[valid_vertices]))
            ma_y.append(pc.mean(yo[valid_vertices],w=wo[valid_vertices]))
            ma_x.append(np.average(xo[valid_vertices],weights=wo[valid_vertices]))
        except:
            pass

    return np.array(ma_x), np.array(ma_y), np.array(ma_y_ci)

def weighted_moving_avg_ecc(x_data,y_data,weights,kernel=6):

    """
    this function was only used in initial version of manuscript
    """
    
    # first sort values based on x
    order = np.argsort(x_data)
    xo = x_data[order]
    yo = y_data[order]
    wo = weights[order]
    n = len(weights)
    kw = int(n/kernel)

    # loop over bins
    # initialize variables
    ma_x = []
    ma_y = []
    ma_y_ci = []    
    for i in range(n-kw):
        valid_vertices = np.arange(len(wo))[i:i+kw]
        valid_vertices = valid_vertices[~np.isnan(wo[valid_vertices])]

        ma_x.append(np.average(xo[valid_vertices],weights=wo[valid_vertices]))
        ma_y.append(np.average(yo[valid_vertices]))#,weights=wo[valid_vertices]))

        # to match the ci with the circular version, find the value that needs to be added
        # and subtracted from the mean value:
        ci = sms.DescrStatsW(yo[valid_vertices],weights=wo[valid_vertices]).tconfint_mean()
        half_ci = np.diff(ci)/2.
        ma_y_ci.append(half_ci[0])
        
#     ma_y_ci = np.array(ma_y_ci)
    
    return np.array(ma_x), np.array(ma_y), np.array(ma_y_ci)

def binned_avg_ang(x_data,y_data,weights,bin_edges):

    """
    This function computes the circular weighted average
    and a 95 CI for the mean
    across y-data for every x-bin given in bin_edges.
    """
    
    # initialize variables
    ma_x = []
    ma_y = []
    ma_y_ci = []
    
    # now loop over bins and compute mean and CI for each bin
    for bi in range(len(bin_edges)-1):
    
        # determine vertices to use for this bin
        # in the last bin, take values including upper limit
        if bi == (len(bin_edges)-1):
            valid_vertices = ((x_data>=bin_edges[bi])*(x_data<=bin_edges[bi+1]))
        # in former bins, take values excluding upper limit
        else:
            valid_vertices = ((x_data>=bin_edges[bi])*(x_data<bin_edges[bi+1]))
        
        valid_verticies = valid_vertices[~np.isnan(valid_vertices)]
            
        # now only compute the values if there are 1 or more valid vertices
        # (this is only relevant in individual subject data)
        if valid_vertices.sum()>0:
            ma_x.append(np.average(x_data[valid_vertices],weights=weights[valid_vertices]))
            ma_y.append(pc.mean(y_data[valid_vertices],w=weights[valid_vertices]))
            ma_y_ci.append(pc.mean_ci_limits(y_data[valid_vertices],w=weights[valid_vertices]))
        else:
            ma_x.append(np.nan)
            ma_y.append(np.nan)
            ma_y_ci.append(np.nan)
    
    return np.array(ma_x), np.array(ma_y), np.array(ma_y_ci)

def binned_avg_ecc(x_data,y_data,weights,bin_edges):

    """
    This function computes the weighted average
    and a 95 CI for the mean
    across y-data for every x-bin given in bin_edges.
    """
    
    # initialize variables
    ma_x = []
    ma_y = []
    ma_y_ci = []

    # now loop over bins and compute mean and CI for each bin
    for bi in range(len(bin_edges)-1):
    
        # determine vertices to use for this bin
        # in the last bin, take values including upper limit
        if bi == (len(bin_edges)-1):
            valid_vertices = ((x_data>=bin_edges[bi])*(x_data<=bin_edges[bi+1]))
        # in former bins, take values excluding upper limit
        else:
            valid_vertices = ((x_data>=bin_edges[bi])*(x_data<bin_edges[bi+1]))

        valid_verticies = valid_vertices[~np.isnan(valid_vertices)]
        # now only compute the values if there are 1 or more valid vertices
        # (this is only relevant in individual subject data)
        if valid_vertices.sum()>3:
            ma_x.append(np.average(x_data[valid_vertices],weights=weights[valid_vertices]))
            ma_y.append(np.average(y_data[valid_vertices],weights=weights[valid_vertices]))
            
            # to match the ci with the circular version, find the value that needs to be added
            # and subtracted from the mean value:
            ci = sms.DescrStatsW(y_data[valid_vertices],weights=weights[valid_vertices]).tconfint_mean()
            half_ci = np.diff(ci)/2.
            ma_y_ci.append(half_ci[0])

        else:
            ma_x.append(np.nan)
            ma_y.append(np.nan)
            ma_y_ci.append(np.nan)
    
    return np.array(ma_x), np.array(ma_y), np.array(ma_y_ci)

def weighted_moving_angle_avg(sj,mask,n_bins,avg_type ='moving'):

    # load in the SUIT data for this subject
    angles=np.ravel(pd.read_csv(os.path.join(flat_dir,'ang_flatdata_%s.csv'%str(sj)),header=None))
    r2=np.ravel(pd.read_csv(os.path.join(flat_dir,'r2_flatdata_%s.csv'%str(sj)),header=None))
    
    # rotate angles so 0 is up and run from 0-2pi
    angles += 1.5*np.pi
    angles = np.mod(angles,np.pi*2)
    
    # create mask
    v = np.zeros_like(masks).astype(bool)
    for subroi in these_combs[mask]:        
        v += (masks==mask_names[subroi])
    
    # rotate x values
    rotation = angle_rot_mask[mask]
    rx = np.ravel(rotate_via_numpy(x,y,rotation*np.pi)[0])
    
    # determine percentile bins:
    sorted_rx = np.sort(copy.copy(rx[v]))
    n = v.sum()
    bin_width = np.int(np.round(n/(n_bins)))
    bin_edge_indices = np.floor(np.linspace(0,n-1,n_bins+1)).astype(int)
    bin_edges = sorted_rx[bin_edge_indices]
       
    # now that bin edges have been defined, add this to mask:
    v *= (r2>0)
    v *= (np.invert(np.isnan(angles)))
    v *= (np.invert(np.isnan(r2)))

    # invert angle data if left hemisphere
    # this is done so both hemispheres show a 'v' shape 
    if 'left' in mask:
        angledata = copy.copy(angles)
        angledata = 2*np.pi-angledata
    elif 'right' in mask:
        angledata = copy.copy(angles)

    # apply mask
    x_data = rx[v]
    y_data = angledata[v]
    weights = r2[v]
    
    # actually compute the binned means
    if avg_type == 'bins':
        ma_x, ma_y, ma_y_ci = binned_avg_ang(x_data,y_data,weights,bin_edges)
    elif avg_type == 'moving':
        ma_x, ma_y, ma_y_ci = weighted_moving_avg_ang(x_data,y_data,weights,n_bins)

    # convert to degrees
    ma_y = np.degrees(ma_y)
    ma_y_ci = np.degrees(ma_y_ci)
    
    return ma_x, ma_y, ma_y_ci

def weighted_moving_ecc_avg(sj,mask,n_bins,avg_type='moving'):

    # get SUIT data for this subject
    eccs=np.ravel(pd.read_csv(os.path.join(flat_dir,'ecc_flatdata_%s.csv'%str(sj)),header=None))
    r2=np.ravel(pd.read_csv(os.path.join(flat_dir,'r2_flatdata_%s.csv'%str(sj)),header=None))
    
    # create mask
    v = np.zeros_like(masks).astype(bool)
    for subroi in these_combs[mask]:        
        v += (masks==mask_names[subroi])
    
    # rotate x values
    rotation = ecc_rot_mask[mask]
    rx = np.ravel(rotate_via_numpy(x,y,rotation*np.pi)[0])
    
    # determine percentile bins:
    sorted_rx = np.sort(copy.copy(rx[v]))
    n = v.sum()
    bin_width = np.int(np.round(n/(n_bins)))
    bin_edge_indices = np.floor(np.linspace(0,n-1,n_bins+1)).astype(int)
    bin_edges = sorted_rx[bin_edge_indices]
   
    # now add this to mask:
    v *= (np.invert(np.isnan(eccs)))
    v *= (np.invert(np.isnan(r2)))
    v *= (r2>0)

    # apply mask
    x_data = rx[v]
    y_data = eccs[v]
    weights = r2[v]
    
    # compute binned means:
    if avg_type == 'moving':
        ma_x, ma_y, ma_y_ci = weighted_moving_avg_ecc(x_data,y_data,weights,n_bins)    
    elif avg_type == 'bins':
        ma_x, ma_y, ma_y_ci = binned_avg_ecc(x_data,y_data,weights,bin_edges)

    
    return ma_x, ma_y, ma_y_ci


# Polar angle plots
### polar progressions for the differently split averaged subjects in separate plots

In [ ]:
pl.close('all')
sjs = ['181','182','183','183_1','183_2']
all_x = {}
all_y = {}
for sj in sjs:
    
    all_x[sj] = {}
    all_y[sj] = {}

    f = pl.figure(figsize=(1.7,2.75))

    alpha = 1
    lw = 1.5
    ls = '-'
    
    for mi,mask in enumerate(roi_order):

        # get data
        ma_x, ma_y,ma_y_ci= weighted_moving_angle_avg(sj,mask,n_bins=10,avg_type='bins')

        all_x[sj][mask] = ma_x
        all_y[sj][mask] = ma_y

        # remove nans
        ma_x = ma_x[~np.isnan(ma_x)]
        ma_y = ma_y[~np.isnan(ma_y)]
        ma_y_ci = ma_y_ci[~np.isnan(ma_y_ci)]

        # now plot
        s = f.add_subplot(3,2,mi+1)
        
        c = get_roi_color(mask)

        pl.fill_between(ma_x,ma_y+ma_y_ci,ma_y-ma_y_ci,color=c,alpha=1)

        # add line at minimal angle
        if 'VIIb' not in mask:
            pl.axvline(ma_x[np.argmin(ma_y)],ls='--',c='k',dashes=(2,2))

        # put down right lables
        # labels are reversed, as data was reversed in left mask
        if 'left' in mask:
            pl.ylim(160,290)
            pl.yticks([180,270],[r'$\pi$',r'$\pi/2$'])#['down','left'])
        elif 'right' in mask:
            pl.ylim(160,290)
            pl.yticks([180,270],[r'$\pi$',r'$3\pi/2$'])#,['down','right'])

        pl.xticks([np.nanmin(ma_x),np.nanmax(ma_x)])
        sn.despine(offset=2)

    pl.tight_layout()
    f.savefig(os.path.join(fig_dir,'anglesurf_cluster_%s.pdf'%sj))

### polar progressions for the differently split averaged subjects in one plot

In [ ]:
pl.close('all')
f = pl.figure(figsize=(1.7,2.75))
lw = 1
ls = '-'
alpha =0.5
sjs = ['181','182','183_1','183_2']

for sj in sjs:

    for mi,mask in enumerate(roi_order):

        # get data
        ma_x, ma_y,ma_y_ci = weighted_moving_angle_avg(sj,mask,n_bins=10,avg_type='bins')

        # plot
        c = get_roi_color(mask)
        s = f.add_subplot(3,2,mi+1)
        pl.fill_between(ma_x,ma_y+ma_y_ci,ma_y-ma_y_ci,color=c,alpha=alpha)

        # add line at minimal angle
        if 'VIIb' not in mask:
            pl.axvline(ma_x[np.nanargmin(ma_y)],lw=1.5,ls='-',c='k', alpha=0.15)

        # put down right lables
        # labels are reversed, as data was reversed in left mask
        if 'left' in mask:
            pl.ylim(160,290)
            pl.yticks([180,270],[r'$\pi$',r'$\pi/2$'])#['down','left'])
        elif 'right' in mask:
            pl.ylim(160,290)
            pl.yticks([180,270],[r'$\pi$',r'$3\pi/2$'])#,['down','right'])

        pl.xticks([np.nanmin(ma_x),np.nanmax(ma_x)])

        sn.despine(offset=2)

pl.tight_layout()
f.savefig(os.path.join(fig_dir,'anglesurf_cluster_together.pdf'))

### polar progressions across individual subjects

In [ ]:
pl.close('all')

# main figure
f = pl.figure(figsize=(1.7,2.75))
# figure with subject Ns
f2 = pl.figure(figsize=(1.7,2.75))

sjs = range(181)

for mi,mask in enumerate(roi_order):
    s = f.add_subplot(3,2,mi+1)
    s2 = f2.add_subplot(3,2,mi+1)

    # initialize variables
    all_xs = []
    all_ys = []

    # now loop over subjects and compute angle
    # for every 10-percentile bin along
    # the cluster extent
    for sj in sjs:
    
        # can't manage for all subjects (too little data)
        try:
            ma_x, ma_y,ma_y_ci = weighted_moving_angle_avg(sj,mask,n_bins=10,avg_type='bins')
            all_xs.append(ma_x)
            all_ys.append(ma_y)            
        except:
            pass
    
    # the x is easy, simply the nanmean across subjects:
    ma_x = np.nanmean(all_xs,axis=0)
    
    # for the y, things are slightly more complicated:
    all_ys = np.array(all_ys)
    ma_y = []
    ma_y_ci = []
    ns = []
    # compute the circular mean and CI of mean for each bin across subjects:
    for b in range(all_ys.shape[1]):
        these_y = all_ys[:,b]
        these_y = these_y[~np.isnan(these_y)]
        ns.append(len(these_y))
        # cannot always compute CI (only if there is certain concentration around mean)
        try:
            ci = np.degrees(pc.mean_ci_limits(np.radians(these_y)))
        except:
            ci = 360 # set to complete uncertainty

        ma_y.append(np.degrees(pc.mean(np.radians(these_y))))
        ma_y_ci.append(ci)
    
    # convert to numpy arrays
    ma_y_ci = np.array(ma_y_ci)
    ma_y = np.array(ma_y)

    # now plot
    c = get_roi_color(mask)
    pl.sca(s)
    pl.plot(ma_x,ma_y,color=c,ls=ls,lw=lw,alpha=1)
    pl.fill_between(ma_x,ma_y-ma_y_ci,ma_y+ma_y_ci,color=c,alpha=0.5)

    # put down right lables
    # labels are reversed, as data was reversed in left mask
    if 'left' in mask:
        pl.ylim(160,290)
        pl.yticks([180,270],[r'$\pi$',r'$\pi/2$'])#['down','left'])
    elif 'right' in mask:
        pl.ylim(160,290)
        pl.yticks([180,270],[r'$\pi$',r'$3\pi/2$'])#,['down','right'])
        
    # ticks
    pl.xticks([np.nanmin(ma_x),np.nanmax(ma_x)])

    # now plot the Ns
    pl.sca(s2)
    pl.plot(ma_x,ns,c=c,lw=2)
    pl.xticks([np.nanmin(ma_x),np.nanmax(ma_x)])
    pl.yticks([0,len(sjs)])

    # and some plot properties
    pl.sca(s)
    sn.despine(offset=2)
    pl.sca(s2)
    sn.despine(offset=2)

# and figure properties
pl.figure(f.number)
pl.tight_layout()
f.savefig(os.path.join(fig_dir,'anglesurf_cluster_together_subs.pdf'))

pl.figure(f2.number)
pl.tight_layout()
f2.savefig(os.path.join(fig_dir,'anglesurf_cluster_together_subs_n.pdf'))


# Eccentricity plots
### eccentricity progressions for the differently split averaged subjects in separate plots

In [ ]:
sjs = ['181','182','183','183_1','183_2']

pl.close('all')
for sj in sjs:
    
    f = pl.figure(figsize=(1.5,2.75))

    for mi,mask in enumerate(roi_order):

        s = f.add_subplot(3,2,mi+1)

        # get data
#         ma_x, ma_y,ma_y_ci = weighted_moving_ecc_avg(sj,mask,n_bins=6,avg_type='moving')
        ma_x, ma_y,ma_y_ci = weighted_moving_ecc_avg(sj,mask,n_bins=8,avg_type='bins')
        # plot
        c = get_roi_color(mask)
#         pl.plot(ma_x,ma_y,lw=1.5,color=c)
        pl.fill_between(ma_x,ma_y+ma_y_ci,ma_y-ma_y_ci,color=c,alpha=1)

        # add ticks
        if (mask == 'left_OMV') + (mask == 'right_OMV'):
            pl.yticks([0,0.5],['0','0.5'])
            pl.ylim(0,0.5)
        elif (mask == 'left_VIIb') + (mask == 'right_VIIb'):
            pl.yticks([0,4],['0','4.0'])
            pl.ylim(0,4)
        elif (mask == 'left_VIIIb') + (mask == 'right_VIIIb'):
            pl.yticks([0,8],['0','8.0'])
            pl.ylim(0,8)
        pl.xticks([np.nanmin(ma_x),np.nanmax(ma_x)])
        sn.despine(offset=2)

    pl.tight_layout()
    f.savefig(os.path.join(fig_dir,'eccsurf_cluster_%s.pdf'%sj))#,dpi=500)

### eccentricity progressions for the differently split averaged subjects in one plot

In [ ]:
pl.close('all')
f = pl.figure(figsize=(1.7,2.75))
lw = 1
ls = '-'        
alpha =0.5


sjs = ['181','182','183_1','183_2']
for sj in sjs:
    

    for mi,mask in enumerate(roi_order):

        #get data
        ma_x, ma_y,ma_y_ci = weighted_moving_ecc_avg(sj,mask,n_bins=8,avg_type='bins')

        # plot
        c = get_roi_color(mask)
        s = f.add_subplot(3,2,mi+1)
        pl.fill_between(ma_x,ma_y+ma_y_ci,ma_y-ma_y_ci,color=c,alpha=alpha)

        # ticks
        if (mask == 'left_OMV') + (mask == 'right_OMV'):
            pl.yticks([0,0.5],['0','0.5'])
            pl.ylim(0,0.5)
        elif (mask == 'left_VIIb') + (mask == 'right_VIIb'):
            pl.yticks([0,4],['0','4.0'])
            pl.ylim(0,4)
        elif (mask == 'left_VIIIb') + (mask == 'right_VIIIb'):
            pl.yticks([0,8],['0','8.0'])
            pl.ylim(0,8)
        pl.xticks([np.nanmin(ma_x),np.nanmax(ma_x)])

        sn.despine(offset=2)

pl.tight_layout()
f.savefig(os.path.join(fig_dir,'eccsurf_cluster_together.pdf'))#,dpi=500)

### polar progressions across individual subjects

In [ ]:
pl.close('all')

# main figure
f = pl.figure(figsize=(1.7,2.75))
# figure with subject Ns
f2 = pl.figure(figsize=(1.7,2.75))

sjs = range(181)

for mi,mask in enumerate(roi_order):
    s = f.add_subplot(3,2,mi+1)
    s2 = f2.add_subplot(3,2,mi+1)

    # initialize variables
    all_xs = []
    all_ys = []

    # now loop over subjects and compute angle
    # for every 10-percentile bin along
    # the cluster extent
    for sj in sjs:
    
        # can't manage for all subjects (too little data)
        try:
            ma_x, ma_y,ma_y_ci = weighted_moving_ecc_avg(sj,mask,n_bins=8,avg_type='bins')
            all_xs.append(ma_x)
            all_ys.append(ma_y)            
        except:
            pass
    
    # the x is easy, simply the nanmean across subjects:
    ma_x = np.nanmean(all_xs,axis=0)
    
    # for the y, things are slightly more complicated:
    all_ys = np.array(all_ys)
    ma_y = []
    ma_y_ci = []
    ns = []
    # compute the circular mean and CI of mean for each bin across subjects:
    for b in range(all_ys.shape[1]):
        these_y = all_ys[:,b]
        these_y = these_y[~np.isnan(these_y)]
        ns.append(len(these_y))
        # cannot always compute CI (only if there is certain concentration around mean)
        try:
            ci = np.degrees(pc.mean_ci_limits(np.radians(these_y)))
        except:
            ci = 360 # set to complete uncertainty

        ma_y.append(np.degrees(pc.mean(np.radians(these_y))))
        ma_y_ci.append(ci)
    
    # convert to numpy arrays
    ma_y_ci = np.array(ma_y_ci)
    ma_y = np.array(ma_y)

    # now plot
    c = get_roi_color(mask)
    pl.sca(s)
    pl.plot(ma_x,ma_y,color=c,ls=ls,lw=lw,alpha=1)
    pl.fill_between(ma_x,ma_y-ma_y_ci,ma_y+ma_y_ci,color=c,alpha=0.5)

    # ticks
    if (mask == 'left_OMV') + (mask == 'right_OMV'):
        pl.yticks([0,2],['0','2'])
        pl.ylim(0,2)
    elif (mask == 'left_VIIb') + (mask == 'right_VIIb'):
        pl.yticks([0,4],['0','4.0'])
        pl.ylim(0,4)
    elif (mask == 'left_VIIIb') + (mask == 'right_VIIIb'):
        pl.yticks([0,8],['0','8.0'])
        pl.ylim(0,8)
        
    # ticks
    pl.xticks([np.nanmin(ma_x),np.nanmax(ma_x)])

    # now plot the Ns
    pl.sca(s2)
    pl.plot(ma_x,ns,c=c,lw=2)
    pl.xticks([np.nanmin(ma_x),np.nanmax(ma_x)])
    pl.yticks([0,len(sjs)])

    # and some plot properties
    pl.sca(s)
    sn.despine(offset=2)
    pl.sca(s2)
    sn.despine(offset=2)

# and figure properties
pl.figure(f.number)
pl.tight_layout()
f.savefig(os.path.join(fig_dir,'eccsurf_cluster_together_subs.pdf'))

pl.figure(f2.number)
pl.tight_layout()
f2.savefig(os.path.join(fig_dir,'eccsurf_cluster_together_subs_n.pdf'))

